## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-31-20-50-30 +0000,nypost,Michael Burry says he’s not shorting Tesla aft...,https://nypost.com/2025/12/31/business/michael...
1,2025-12-31-20-48-12 +0000,bbc,Congress releases Jack Smith's testimony about...,https://www.bbc.com/news/articles/c0mkn2ylv3zo...
2,2025-12-31-20-45-27 +0000,nypost,ICE will no longer hold detainees at LI courth...,https://nypost.com/2025/12/31/us-news/ice-will...
3,2025-12-31-20-45-00 +0000,wsj,Saks Prepares for Bankruptcy After Missing Deb...,https://www.wsj.com/finance/saks-prepares-for-...
4,2025-12-31-20-44-14 +0000,bbc,In pictures: New Year welcomed around the world,https://www.bbc.com/news/articles/c050j896q2do...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_3027/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,year,33
14,trump,24
32,new,22
423,ukraine,8
17,will,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
228,2025-12-30-23-05-25 +0000,cbc,Israel will ban dozens of aid groups from Gaza...,https://www.cbc.ca/radio/asithappens/gaza-aid-...,95
129,2025-12-31-11-05-20 +0000,nypost,Washington Monument will glow on New Year’s Ev...,https://nypost.com/2025/12/31/us-news/washingt...,85
55,2025-12-31-18-19-37 +0000,nyt,"In New Year’s Speech to Russia, Putin Says Lit...",https://www.nytimes.com/2025/12/31/world/europ...,80
42,2025-12-31-19-02-59 +0000,nypost,NJ drivers hit with 9% state gas tax hike to k...,https://nypost.com/2025/12/31/us-news/nj-drive...,79
198,2025-12-31-02-27-00 +0000,wsj,The Trump administration said it is freezing f...,https://www.wsj.com/politics/policy/trump-admi...,72


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
228,95,2025-12-30-23-05-25 +0000,cbc,Israel will ban dozens of aid groups from Gaza...,https://www.cbc.ca/radio/asithappens/gaza-aid-...
198,72,2025-12-31-02-27-00 +0000,wsj,The Trump administration said it is freezing f...,https://www.wsj.com/politics/policy/trump-admi...
43,41,2025-12-31-19-02-13 +0000,nypost,DOJ ‘working around the clock’ on Epstein file...,https://nypost.com/2025/12/31/us-news/doj-work...
59,37,2025-12-31-17-56-58 +0000,nypost,OpenAI’s pay tops every major tech startup as ...,https://nypost.com/2025/12/31/business/openais...
57,33,2025-12-31-18-01-00 +0000,wsj,Trump’s surprise veto of a Colorado water proj...,https://www.wsj.com/politics/policy/trump-veto...
205,31,2025-12-31-01-49-23 +0000,nypost,Billionaire Bill Ackman warns ‘no one would st...,https://nypost.com/2025/12/30/us-news/californ...
102,31,2025-12-31-14-48-14 +0000,nypost,"Russian drone attack injures 6, including chil...",https://nypost.com/2025/12/31/world-news/russi...
37,30,2025-12-31-19-33-00 +0000,nypost,23-year-old drug queenpin who offered up victi...,https://nypost.com/2025/12/31/us-news/23-year-...
145,28,2025-12-31-10-30-00 +0000,wsj,Instead of crumbling under the highest U.S. ta...,https://www.wsj.com/economy/trade/trump-trade-...
174,27,2025-12-31-05-20-55 +0000,nyt,"Tatiana Schlossberg, Kennedy Daughter Who Wrot...",https://www.nytimes.com/2025/12/30/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
